this project will use an already build pedestrian detection module and will update the dataset via new detections, meaning that everytime we detect a pedestrain the chosen pedestrain will be cropped and resized so that the AI module can detect pedestrains of even smaller resolutions as well as much bigger ones, we are doing this because we noticed that sometimes most modules don't detect the especially smaller pedestrains in various images.

In [1]:
  !git clone https://github.com/pjreddie/darknet.git
  %cd darknet
  !make

Cloning into 'darknet'...
remote: Enumerating objects: 5955, done.
remote: Total 5955 (delta 0), reused 0 (delta 0), pack-reused 5955
Receiving objects: 100% (5955/5955), 6.37 MiB | 18.64 MiB/s, done.
Resolving deltas: 100% (3932/3932), done.
/content/darknet
mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/utils.c -o obj/utils.o
In file included from /usr/include/string.h:495,
                 from ./src/utils.c:3:
In function ‘strncpy’,
    inlined from ‘copy_string’ at ./src/utils.c:426:5:
/usr/include/x86_64-linux-gnu/bits/string_fortified.h:106:10: warning: ‘__builtin_strncpy’ specified bound depends on the length of the source argument [-Wstringop-overflow=]
  106 |   return __builtin___strncpy_chk (__dest, __src, __len, __bos (__dest));
      |          ^

In [2]:
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg


--2023-03-24 20:59:35--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  40.5MB/s    in 6.2s    

2023-03-24 20:59:41 (37.9 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2023-03-24 20:59:42--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
ERROR: cannot verify raw.githubusercontent.com's certificate, issued by ‘CN=DigiCert TLS RSA SHA256 2020 CA1,O=DigiCert Inc,C=US’:
  Issued certificate has expired.
To connect to raw.gi

In [5]:
import cv2

# Load the YOLO model
net = cv2.dnn.readNetFromDarknet('/content/darknet/cfg/yolov3.cfg', '/content/darknet/yolov3.weights')

# Set the backend and target to use CUDA (if available)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)


In [21]:
import cv2
import numpy as np
import os

# Function to add salt and pepper noise to image
def add_salt_pepper_noise(img):
    # Randomly add salt and pepper noise to image
    noise = np.zeros_like(img)
    cv2.randu(noise, 0, 255)
    black = noise < 50
    white = noise > 205
    img[black] = 0
    img[white] = 255
    return img


# Load YOLOv3 weights and configuration files
net = cv2.dnn.readNetFromDarknet('/content/darknet/cfg/yolov3.cfg', '/content/darknet/yolov3.weights')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

direct = "/content/darknet/obj.data"
if not os.path.isfile(direct):
    with open(direct, 'w') as f:
        f.write('classes = 1\n')
        f.write('train = /content/darknet/train.txt\n')
        f.write('valid = /content/darknet/test.txt\n')
        f.write('names = /content/darknet/data/obj.names\n')
        f.write('backup = /content/drive/MyDrive/yolov3/backup/\n')

# Load pedestrian detection classes
with open('/content/darknet/data/coco.names', 'rt') as f:
    pedestrian_classes = f.read().rstrip('\n').split('\n')
    pedestrian_classes = [c for c in pedestrian_classes if 'person' in c]

# Load image
#please replace the current path with the path of the image you wish to run through the AI module.
img = cv2.imread('/content/darknet/data/person.jpg')
height, width, _ = img.shape

# Create blob from image
blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), swapRB=True, crop=False)

# Set input for YOLOv3 network
net.setInput(blob)

# Run forward pass of YOLOv3 network
outs = net.forward(net.getUnconnectedOutLayersNames())

# Extract bounding boxes, confidences, and class IDs
boxes = []
confidences = []
class_ids = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5 and class_id < len(pedestrian_classes) and pedestrian_classes[class_id]:
            center_x, center_y, w, h = (detection[:4] * np.array([width, height, width, height])).astype('int')
            x, y = int(center_x - w / 2), int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply non-max suppression to remove overlapping boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Save cropped pedestrian images and draw bounding boxes on original image
if len(indices) > 0:
    for i in range(len(indices)):
        if len(indices) > 0:
            for index in indices:
                x, y, w, h = boxes[index]
                pedestrian_img = img[y:y+h, x:x+w]

                # Resize the pedestrian_img to smaller and larger sizes
                small_img = cv2.resize(pedestrian_img, (100, 100))
                large_img = cv2.resize(pedestrian_img, (500, 500))
                directory = "/content/darknet/dataset"
                # Check if directory exists
                if not os.path.exists(directory):
                  # Create the directory if it does not exist
                  os.makedirs(directory)
                  
                # Save the resized images to the directory
                cv2.imwrite(f"/content/darknet/dataset/pedestrian_{index}_small.jpg", small_img)
                cv2.imwrite(f"/content/darknet/dataset/pedestrian_{index}_large.jpg", large_img)
                with open("train.txt", "a") as f:
                  f.write("/content/darknet/dataset/pedestrian_{index}_small.jpg \n")
                  f.write("/content/darknet/dataset/pedestrian_{index}_large.jpg \n")
                 # Add salt and pepper noise to small and large images
                small_img_noisy = add_salt_pepper_noise(small_img)
                large_img_noisy = add_salt_pepper_noise(large_img)
                # Save the resized images to the directory
                cv2.imwrite(f"/content/darknet/dataset/pedestrian_{index}_small_SP.jpg", small_img_noisy)
                cv2.imwrite(f"/content/darknet/dataset/pedestrian_{index}_large_SP.jpg", large_img_noisy)

                with open("train.txt", "a") as f:
                  f.write("/content/darknet/dataset/pedestrian_{index}_small_SP.jpg \n")
                  f.write("/content/darknet/dataset/pedestrian_{index}_large_SP.jpg \n")
                # Draw bounding boxes on original image
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Save output image with bounding boxes
    cv2.imwrite('output_image.jpg', img)

else:
    print("No pedestrians detected in the input image.")


Train the AI module after identifying and saving the images

In [22]:
!./darknet detector train obj.data cfg/yolov3.cfg darknet53.conv.74
#reset the contents of the train.txt so that we dont overtrain the module with the same images
#basically after training we delete the images waiting for the next run
with open('/content/darknet/train.txt', 'w') as f:
    f.write('')

yolov3
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                